In [ ]:
import random
import os
import numpy as np
import sklearn
import torch
from torch.cuda import manual_seed_all
import matplotlib as mpl
from matplotlib import pyplot as plt
import torch.nn as nn

In [ ]:
# DenseNet Model
import torchvision

class densenet(torch.nn.Module):
    """
    DenseNet Class, derived from Pytorch. Intended for model manipulation (i.e. unfreezing layers, etc.)
    To use model, try (densenet).model(data)
    May change to reflect manual implementation of densenet161.
    """
    def __init__(self):
        super().__init__()  # Initialize the nn.Module base class
        self.model = torchvision.models.densenet161()

    def forward(self, x):
        return self.model(x)  # Delegate forward pass to the original DenseNet

    def layer_change(self):
        """
        Unfreeze layers of DenseNet model per specifications
        """
        pass

In [ ]:
# Define training and testing loops
def train_loop(dataloader, model, loss_fn, optimizer, epochs=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        size = len(dataloader.dataset)
        total_loss = 0  # Initialize variable to accumulate loss per epoch

        for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss
            total_loss += loss.item()

            if batch % 20 == 0:
                current = (batch + 1) * len(X)
                print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

        # Average loss for this epoch
        avg_loss = total_loss / len(dataloader)
        print(f"Training Loss (Epoch): {avg_loss:>7f}")
    return avg_loss  # Return the average loss for the last epoch


def test_loop(dataloader, model, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Average loss and accuracy for this fold
    avg_test_loss = test_loss / num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {avg_test_loss:>8f} \n")
    return avg_test_loss, accuracy  # Return both average loss and accuracy for this fold

In [7]:
# Model Testing
test_mod = densenet()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(test_mod.model.parameters(), lr=0.0001)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    #Run training loop
    test_mod.model.train()
    train_loop(train_dataloader, test_mod.model, loss_fn, optimizer)
    test_mod.model.eval()

    # Test on Validation set, retrieve loss
    test_loop(val_dataloader, test_mod.model, loss_fn)

Epoch 1
-------------------------------


NameError: name 'train_dataloader' is not defined